# Lab 02: Vanilla RNN yourself - exercise

In [ ]:
# For Google Colaboratory
import sys, os
if 'google.colab' in sys.modules:
    # mount google drive
    from google.colab import drive
    drive.mount('/content/gdrive')
    path_to_file = '/content/gdrive/My Drive/CS5242_2025_codes/labs_lecture06/lab02_vrnn_yourself'
    print(path_to_file)
    # change current path to the folder containing "file_name"
    os.chdir(path_to_file)
    !pwd

In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import math
import time
import utils

### With or without GPU

It is recommended to run this code on GPU:<br> 
* Time for 1 epoch on CPU : 153 sec ( 2.55 min)<br> 
* Time for 1 epoch on GPU : 8.4 sec w/ GeForce GTX 1080 Ti <br>

In [2]:
device= torch.device("cuda")
device= torch.device("cpu")
print(device)

if torch.cuda.is_available():
    print('cuda available with GPU:',torch.cuda.get_device_name(0))

cpu


### Download Penn Tree Bank

The tensor train_data consists of 20 columns of 46,479 words.<br>
The tensor test_data consists of 20 columns of 4,121 words.

In [3]:
from utils import check_ptb_dataset_exists
data_path=check_ptb_dataset_exists()

train_data  =  torch.load(data_path+'ptb/train_data.pt')
test_data   =  torch.load(data_path+'ptb/test_data.pt')

print(  train_data.size()  )
print(  test_data.size()   )

torch.Size([46479, 20])
torch.Size([4121, 20])


### Some constants associated with the data set

In [4]:
bs = 20

vocab_size = 10000

### Make a recurrent net class

Implement the vanilla RNN network without PyTorch:

$$
h_t = \tanh ( Rh_{t-1} + V g_t)
$$


In [ ]:
class my_VRNN(nn.Module):
    
    def __init__(self, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.R = nn.Linear(hidden_size, hidden_size)
        self.V = nn.Linear(hidden_size, hidden_size)
        # COMPLETE HERE 
    
    
    def forward(self, g_seq, h_init):
        # g_seq: (T, B, H)
        # h_init: (1, B, H)
        t = len(g_seq)           # t: Sequence length
        h_seq_list = []          # Will hold T tensors of shape [1, B, H]
        h_prev = h_init          # h_prev: [1, B, H]
        for i in range(t):
            h_current = torch.tanh(self.R(h_prev) + self.V(g_seq[i]))   # [1, B, H] + [B, H] -> [1, B, H] pytorch broadcasting see next cell
            h_seq_list.append(h_current)
            h_prev = h_current  # [1, B, H]
        h_seq = torch.cat(h_seq_list) # [T, B, H]  not if it is torch.stack, then shape is [T, 1, B, H]
        h_final = h_current     # [1, B, H]
        return h_seq, h_final

#answer
class my_VRNN_answer(nn.Module):
    
    def __init__(self, hidden_size):
        super().__init__()
        self.R = nn.Linear( hidden_size , hidden_size )
        self.V = nn.Linear( hidden_size , hidden_size )
        
    # TA use cat because it already has an extra dimension
    def forward(self, g_seq , h_init ):
        # g_seq:  (T, B, H)
        # h_init: (1, B, H)
        V_g_all_t = self.V(g_seq) # (T, B, H)
        h_t_pre = h_init    # (1, B, H)
        h_seq = []
        for V_g_t in V_g_all_t:
            R_h_t_pre = self.R(h_t_pre)  # (1, B, H) -> (1, B, H)
            h_t = torch.tanh( R_h_t_pre + V_g_t ) # (1, B, H) + (B, H) -> (1, B, H) due to broadcasting
            h_seq.append(h_t)
            h_t_pre = h_t
        h_seq = torch.cat(h_seq) 
        # h_seq[-1,:,:] is the same as h_seq[-1] and is (B, H) and we need to add an extra dimension to make it (1, B, H)
        h_final = h_seq[-1,:,:].unsqueeze(0)
        return h_seq , h_final
    
class three_layer_recurrent_net(nn.Module):

    def __init__(self, hidden_size):
        super(three_layer_recurrent_net, self).__init__()
        
        self.layer1 = nn.Embedding( vocab_size  , hidden_size  )
        self.layer2 = my_VRNN( hidden_size )
        self.layer3 = nn.Linear(    hidden_size , vocab_size   )

        
    def forward(self, word_seq, h_init ):
        
        g_seq               =   self.layer1( word_seq )  
        h_seq , h_final     =   self.layer2( g_seq , h_init )
        score_seq           =   self.layer3( h_seq )
        
        return score_seq,  h_final 


In [4]:
# TA's code also works because of pytorch's broadcasting
a = torch.rand(1,2,3)
b = torch.rand(2,3)
c = a+b
d = a + b.unsqueeze(0)
print(a.size(), b.size(), c.size(), d.size())
c == d

torch.Size([1, 2, 3]) torch.Size([2, 3]) torch.Size([1, 2, 3]) torch.Size([1, 2, 3])


tensor([[[True, True, True],
         [True, True, True]]])

### Build the net. Choose the hidden size to be 150. How many parameters in total?

In [15]:
hidden_size=150

net = three_layer_recurrent_net( hidden_size )

print(net)

utils.display_num_param(net)

three_layer_recurrent_net(
  (layer1): Embedding(10000, 150)
  (layer2): my_VRNN(
    (R): Linear(in_features=150, out_features=150, bias=True)
    (V): Linear(in_features=150, out_features=150, bias=True)
  )
  (layer3): Linear(in_features=150, out_features=10000, bias=True)
)
There are 3055300 (3.06 million) parameters in this neural network


### Send the weights of the networks to the GPU

In [16]:
net = net.to(device)

### Set up manually the weights of the embedding module and Linear module

In [17]:
net.layer1.weight.data.uniform_(-0.1, 0.1)

net.layer3.weight.data.uniform_(-0.1, 0.1)

print('')

### Choose the criterion, as well as the following important hyperparameters: 
* initial learning rate = 1
* sequence length = 35

In [18]:
criterion = nn.CrossEntropyLoss()

my_lr = 1

seq_length = 35

### Function to evaluate the network on the test set

In [19]:
def eval_on_test_set():

    running_loss=0
    num_batches=0    
       
    h = torch.zeros(1, bs, hidden_size)
    
    h=h.to(device)

       
    for count in range( 0 , 4120-seq_length ,  seq_length) :
               
        minibatch_data =  test_data[ count   : count+seq_length   ]
        minibatch_label = test_data[ count+1 : count+seq_length+1 ]
        
        minibatch_data=minibatch_data.to(device)
        minibatch_label=minibatch_label.to(device)
                                  
        scores, h  = net( minibatch_data, h )
        
        minibatch_label =   minibatch_label.view(  bs*seq_length ) 
        scores          =            scores.view(  bs*seq_length , vocab_size)
        
        loss = criterion(  scores ,  minibatch_label )    
        
        h=h.detach()
            
        running_loss += loss.item()
        num_batches += 1        
    
    total_loss = running_loss/num_batches 
    print('test: exp(loss) = ', math.exp(total_loss)  )
        

### Do 10 passes through the training set (100 passes would reach 135 on test set)

In [20]:
start=time.time()

for epoch in range(10):
    
    # keep the learning rate to 1 during the first 4 epochs, then divide by 1.1 at every epoch
    if epoch >= 4:
        my_lr = my_lr / 1.1
    
    # create a new optimizer and give the current learning rate.   
    optimizer=torch.optim.SGD( net.parameters() , lr=my_lr )
        
    # set the running quantities to zero at the beginning of the epoch
    running_loss=0
    num_batches=0    
       
    # set the initial h to be the zero vector
    h = torch.zeros(1, bs, hidden_size)

    # send it to the gpu    
    h=h.to(device)
    
    for count in range( 0 , 46478-seq_length ,  seq_length):
             
        # Set the gradients to zeros
        optimizer.zero_grad()
        
        # create a minibatch
        minibatch_data =  train_data[ count   : count+seq_length   ]
        minibatch_label = train_data[ count+1 : count+seq_length+1 ]        
        
        # send them to the gpu
        minibatch_data=minibatch_data.to(device)
        minibatch_label=minibatch_label.to(device)
        
        # Detach to prevent from backpropagating all the way to the beginning
        # Then tell Pytorch to start tracking all operations that will be done on h and c
        h=h.detach()
        h=h.requires_grad_()
                       
        # forward the minibatch through the net        
        scores, h  = net( minibatch_data, h )
        
        # reshape the scores and labels to huge batch of size bs*seq_length
        scores          =            scores.view(  bs*seq_length , vocab_size)  
        minibatch_label =   minibatch_label.view(  bs*seq_length )       
        
        # Compute the average of the losses of the data points in this huge batch
        loss = criterion(  scores ,  minibatch_label )
        
        # backward pass to compute dL/dR, dL/dV and dL/dW
        loss.backward()

        # do one step of stochastic gradient descent: R=R-lr(dL/dR), V=V-lr(dL/dV), ...
        utils.normalize_gradient(net)
        optimizer.step()
        
        # update the running loss  
        running_loss += loss.item()
        num_batches += 1
        
        
        
    # compute stats for the full training set
    total_loss = running_loss/num_batches
    elapsed = time.time()-start
    
    print('')
    print('epoch=',epoch, '\t time=', elapsed,'\t lr=', my_lr, '\t exp(loss)=',  math.exp(total_loss))
    eval_on_test_set() 



epoch= 0 	 time= 276.6997301578522 	 lr= 1 	 exp(loss)= 523.0824489593288
test: exp(loss) =  333.72204578861573

epoch= 1 	 time= 522.1988043785095 	 lr= 1 	 exp(loss)= 269.7785354066164
test: exp(loss) =  251.1308952285022

epoch= 2 	 time= 789.6021013259888 	 lr= 1 	 exp(loss)= 209.4903977679223
test: exp(loss) =  211.92724755893207

epoch= 3 	 time= 1052.75337266922 	 lr= 1 	 exp(loss)= 178.81547397881758
test: exp(loss) =  194.9606635370571

epoch= 4 	 time= 1212.4742691516876 	 lr= 0.9090909090909091 	 exp(loss)= 157.38269526974233
test: exp(loss) =  182.89685679366127

epoch= 5 	 time= 1336.0235283374786 	 lr= 0.8264462809917354 	 exp(loss)= 142.5764237995476
test: exp(loss) =  172.2976469233339

epoch= 6 	 time= 1457.3349962234497 	 lr= 0.7513148009015777 	 exp(loss)= 131.9495524784308
test: exp(loss) =  166.1694411948323

epoch= 7 	 time= 1642.4559769630432 	 lr= 0.6830134553650705 	 exp(loss)= 123.662302369137
test: exp(loss) =  161.84292280791465

epoch= 8 	 time= 1848.83580

### Choose one sentence (taken from the test set)

In [21]:
sentence1 = "some analysts expect oil prices to remain relatively"

sentence2 = "over the next days and weeks they say investors should look for stocks to"

sentence3 = "prices averaging roughly $ N a barrel higher in the third"

sentence4 = "i think my line has been very consistent mrs. hills said at a news"

sentence5 = "this appears particularly true at gm which had strong sales in"

# or make your own sentence.  No capital letter or punctuation allowed. Each word must be in the allowed vocabulary.
sentence6= "he was very"

# SELECT THE SENTENCE HERE
mysentence = sentence3

### Convert the sentence into a vector, then send to GPU

In [25]:
minibatch_data=utils.sentence2vector(mysentence)
      
minibatch_data=minibatch_data.to(device)

print(minibatch_data)

tensor([[1786],
        [8705],
        [3246],
        [ 416],
        [  27],
        [  35],
        [2664],
        [ 209],
        [ 108],
        [  32],
        [3017]])


### Set the initial hidden state to zero, then run the RNN.

In [ ]:
h = torch.zeros(1, 1, hidden_size)
# h = torch.zeros(1, hidden_size)

h=h.to(device)

scores , h = net( minibatch_data , h )

### Display the network prediction for the next word

In [33]:
print(mysentence, '... \n')

utils.show_next_word(scores)

prices averaging roughly $ N a barrel higher in the third ... 

88.6%	 quarter
1.5%	 period
1.1%	 year
0.7%	 of
0.4%	 consecutive
0.4%	 month
0.3%	 term
0.3%	 <eos>
0.3%	 world
0.3%	 fiscal
0.3%	 and
0.3%	 market
0.2%	 session
0.2%	 area
0.2%	 day
0.2%	 rate
0.1%	 buy-out
0.1%	 week
0.1%	 sector
0.1%	 half
0.1%	 third
0.1%	 floor
0.1%	 time
0.1%	 level
0.1%	 nine
0.1%	 on
0.1%	 section
0.1%	 game
0.1%	 1980s
0.0%	 industry
